In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.model_selection import train_test_split
from joblib import Parallel, delayed
from itertools import repeat
from time import time
from copy import deepcopy
from sklearn.metrics import confusion_matrix

### Already Pre-processed

In [2]:
df2 = pd.read_csv("data/processed_20210622.gz")

In [3]:
df2.shape

(111897, 201)

In [4]:
data1p = pd.read_csv("data/christine_b2f05.gz")
data2p = pd.read_csv("data/christine_b2f10.gz")
data = pd.concat((data1p, data2p))

### With pre-processing

In [2]:
df = pd.read_csv("data/my_20210622.gz")
df.shape

(223810, 353)

In [176]:
np.unique(df.evidencelevel.values, return_counts=True)

(array([1, 2, 3, 4]), array([166028,   5622,   1182,  50978]))

In [177]:
np.sum(df['el4-learn']==1)

41068

In [178]:
df.columns[:17]

Index(['id', 'cl_start', 'cl_len', 'evidencelevel', 'cl_name', 'sequence',
       'kingdom', 'species', 'evidencelevelreeval', 'cas_count', 'cas_class',
       'seq_len', 'cas_prox_dist', 'cas_prox', 'cas_prox_class', 'el4-learn',
       'blastscore'],
      dtype='object')

### First Pre-processing

In [179]:
def move_to_end(df, col_name):
    col = df[col_name]
    df.pop(col_name)
    idx = df.shape[1]
    df.insert(idx, col_name, col)

def move_to_begining(df, col_name):
    col = df[col_name]
    df.pop(col_name)
    df.insert(0, col_name, col)

In [288]:
rnafold_params = ['rnafold_37_energy', 'rnafold_37_frequence', 'rnafold_37_diversity',
       'rnafold_37_stem', 'rnafold_37_stem_diff', 'rnafold_37_loop',
       'rnafold_37_stem_loop_nbr', 'rnafold_75_energy', 'rnafold_75_frequence',
       'rnafold_75_diversity', 'rnafold_75_stem', 'rnafold_75_stem_diff',
       'rnafold_75_loop', 'rnafold_75_stem_loop_nbr', 'rnafold_37_dot_par', 'rnafold_75_dot_par']
# move to end all rnafold parameters
for param in rnafold_params:
    move_to_end(df, param)
    
# re-organize columns s.t. 3mers and 4mers are in alphabetical order
info = df.columns[:17].tolist()
sorted_3mers = sorted(df.columns[17:81])
sorted_4mers = sorted(df.columns[81:337])
rnafolds = df.columns[337:].tolist()

df = df[info+sorted_3mers+sorted_4mers+rnafolds]

In [289]:
df.head()

,id,cl_start,cl_len,evidencelevel,cl_name,sequence,kingdom,species,evidencelevelreeval,cas_count,...,rnafold_37_stem_loop_nbr,rnafold_75_energy,rnafold_75_frequence,rnafold_75_diversity,rnafold_75_stem,rnafold_75_stem_diff,rnafold_75_loop,rnafold_75_stem_loop_nbr,rnafold_37_dot_par,rnafold_75_dot_par
0,00000e27-d9b7-4212-908c-631c344833fd,2222259,129,1,CP065590_1,TGAAAACAACGGTTGTCCTTGGCCAGATACAGACGGTGACGGT,Bacteria,Chryseobacterium indologenes,NaN,1,...,1,0.0,0.267563,6.34,0,0,0,0,........((.(((..((.((........))...)).))).)),...........................................
1,00006819-e6c5-4a18-8f95-c286baf30f9a,848776,100,1,CP022336_1,ACCGCTTCCTGAAATTGCTTCTGGTTCTT,Bacteria,Sphingorhabdus sp. SMR4y,NaN,0,...,1,0.0,0.741981,1.57,0,0,0,0,.......((.(((.....))).)).....,.............................
2,00008500-4136-445e-b0b4-aa0843c1b5b0,279177,94,1,CP022427_2,TATTATTTGACCGCTGTTGCGCC,Bacteria,Pantoea ananatis,NaN,0,...,1,0.0,0.702724,1.26,0,0,0,0,...........(((....)))..,.......................
3,0001edc2-5dc5-44c3-92f1-3d062639ec81,788252,97,1,CP009513_2,TACTCTCCTGTTAAATAAAGAAAGAT,Archaea,Methanosarcina mazei,NaN,2,...,0,0.0,0.872185,0.77,0,0,0,0,..........................,..........................
4,00025e31-c0ea-49bd-a877-b0d440958122,2126067,84,1,CP012022_3,GTACCACCAGCCGCCGTGCCGCCAG,Bacteria,Corynebacterium pseudotuberculosis,NaN,1,...,1,0.0,0.819555,0.77,0,0,0,0,.........((.((...)).))...,.........................


Info

In [290]:
df.columns[:17]

Index(['id', 'cl_start', 'cl_len', 'evidencelevel', 'cl_name', 'sequence',
       'kingdom', 'species', 'evidencelevelreeval', 'cas_count', 'cas_class',
       'seq_len', 'cas_prox_dist', 'cas_prox', 'cas_prox_class', 'el4-learn',
       'blastscore'],
      dtype='object')

3-mers

In [291]:
df.columns[17:81]

Index(['AAA', 'AAC', 'AAG', 'AAT', 'ACA', 'ACC', 'ACG', 'ACT', 'AGA', 'AGC',
       'AGG', 'AGT', 'ATA', 'ATC', 'ATG', 'ATT', 'CAA', 'CAC', 'CAG', 'CAT',
       'CCA', 'CCC', 'CCG', 'CCT', 'CGA', 'CGC', 'CGG', 'CGT', 'CTA', 'CTC',
       'CTG', 'CTT', 'GAA', 'GAC', 'GAG', 'GAT', 'GCA', 'GCC', 'GCG', 'GCT',
       'GGA', 'GGC', 'GGG', 'GGT', 'GTA', 'GTC', 'GTG', 'GTT', 'TAA', 'TAC',
       'TAG', 'TAT', 'TCA', 'TCC', 'TCG', 'TCT', 'TGA', 'TGC', 'TGG', 'TGT',
       'TTA', 'TTC', 'TTG', 'TTT'],
      dtype='object')

4-mers

In [292]:
df.columns[81:337]

Index(['AAAA', 'AAAC', 'AAAG', 'AAAT', 'AACA', 'AACC', 'AACG', 'AACT', 'AAGA',
       'AAGC',
       ...
       'TTCG', 'TTCT', 'TTGA', 'TTGC', 'TTGG', 'TTGT', 'TTTA', 'TTTC', 'TTTG',
       'TTTT'],
      dtype='object', length=256)

rnafold

In [293]:
df.columns[337:351]

Index(['rnafold_37_energy', 'rnafold_37_frequence', 'rnafold_37_diversity',
       'rnafold_37_stem', 'rnafold_37_stem_diff', 'rnafold_37_loop',
       'rnafold_37_stem_loop_nbr', 'rnafold_75_energy', 'rnafold_75_frequence',
       'rnafold_75_diversity', 'rnafold_75_stem', 'rnafold_75_stem_diff',
       'rnafold_75_loop', 'rnafold_75_stem_loop_nbr'],
      dtype='object')

In [294]:
df.columns[351:]

Index(['rnafold_37_dot_par', 'rnafold_75_dot_par'], dtype='object')

### Data Cleaning

Removal of observations with letters K,M,R,Y

In [295]:
np.unique(df.id).size

111905

In [296]:
def is_reverse_complement(seq1, seq2):
    rc = seq2[::-1]
    rc = rc.replace('A', '*')
    rc = rc.replace('T', 'A')
    rc = rc.replace('*', 'T')
    rc = rc.replace('C', '*')
    rc = rc.replace('G', 'C')
    rc = rc.replace('*', 'G')
    return seq1 == rc

In [298]:
# each i-th element (i=0:df.shape[0]/2) is a DR, while each (i+df.shape[0]/2)-th is the corresponding RC
n_obs = int(df.shape[0]/2)
# check is not perfect because there are some DNAs with letters K,M,R,Y
check = np.array([is_reverse_complement(*df.iloc[[i, i+n_obs]]['sequence']) for i in range(n_obs)])
check.sum()

111897

In [299]:
# remove these weird observations
to_delete = np.where(np.logical_not(check))[0]
to_delete = to_delete.tolist() + (to_delete+n_obs).tolist()
df = df.drop(index=to_delete)
df.head()

,id,cl_start,cl_len,evidencelevel,cl_name,sequence,kingdom,species,evidencelevelreeval,cas_count,...,rnafold_37_stem_loop_nbr,rnafold_75_energy,rnafold_75_frequence,rnafold_75_diversity,rnafold_75_stem,rnafold_75_stem_diff,rnafold_75_loop,rnafold_75_stem_loop_nbr,rnafold_37_dot_par,rnafold_75_dot_par
0,00000e27-d9b7-4212-908c-631c344833fd,2222259,129,1,CP065590_1,TGAAAACAACGGTTGTCCTTGGCCAGATACAGACGGTGACGGT,Bacteria,Chryseobacterium indologenes,NaN,1,...,1,0.0,0.267563,6.34,0,0,0,0,........((.(((..((.((........))...)).))).)),...........................................
1,00006819-e6c5-4a18-8f95-c286baf30f9a,848776,100,1,CP022336_1,ACCGCTTCCTGAAATTGCTTCTGGTTCTT,Bacteria,Sphingorhabdus sp. SMR4y,NaN,0,...,1,0.0,0.741981,1.57,0,0,0,0,.......((.(((.....))).)).....,.............................
2,00008500-4136-445e-b0b4-aa0843c1b5b0,279177,94,1,CP022427_2,TATTATTTGACCGCTGTTGCGCC,Bacteria,Pantoea ananatis,NaN,0,...,1,0.0,0.702724,1.26,0,0,0,0,...........(((....)))..,.......................
3,0001edc2-5dc5-44c3-92f1-3d062639ec81,788252,97,1,CP009513_2,TACTCTCCTGTTAAATAAAGAAAGAT,Archaea,Methanosarcina mazei,NaN,2,...,0,0.0,0.872185,0.77,0,0,0,0,..........................,..........................
4,00025e31-c0ea-49bd-a877-b0d440958122,2126067,84,1,CP012022_3,GTACCACCAGCCGCCGTGCCGCCAG,Bacteria,Corynebacterium pseudotuberculosis,NaN,1,...,1,0.0,0.819555,0.77,0,0,0,0,.........((.((...)).))...,.........................


#### Removal of repetitive ids

1. In the data set, for each DR a reverse complement (RC) was generated.
2. To construct canonical k-mers, a DR and its RC is used (AAA_new = AAA_dr + AAA_rc). Then, the sample size will be n/2.

In [300]:
def canonical_mers(dir_repeat, rev_compl, start, end):
    final_example = rev_compl.copy() if rev_compl.rnafold_37_energy < dir_repeat.rnafold_37_energy else dir_repeat.copy()
    final_example.iloc[start:end] = dir_repeat.iloc[start:end] + rev_compl.iloc[start:end]
    return final_example

In [301]:
n_obs = int(df.shape[0] / 2)

In [317]:
df2 = pd.concat([canonical_mers(df.iloc[i], df.iloc[i+n_obs], 17, 337) for i in range(n_obs)], axis=1).T

In [318]:
df2[["TCA", "TGA"]]

,TCA,TGA
0,0.0487805,0.0487805
1,0.037037,0.037037
111907,0.047619,0.047619
111908,0,0
111909,0,0
...,...,...
223805,0.05,0.05
111901,0.136364,0.136364
223807,0.0909091,0.0909091
223808,0.0384615,0.0384615


#### Delete mers that are reverse complements to other mers

In [304]:
def reverse_complement(seq):
    rc = seq[::-1]
    rc = rc.replace('A', '*')
    rc = rc.replace('T', 'A')
    rc = rc.replace('*', 'T')
    rc = rc.replace('C', '*')
    rc = rc.replace('G', 'C')
    rc = rc.replace('*', 'G')
    return rc

In [305]:
def find_sufficient_mers(mers):
    res = []
    for mer in mers:
        rc = reverse_complement(mer)
        if rc == mer:
            res.append(mer)
        else:
            if rc not in res:
                res.append(mer)
    return res

In [321]:
to_keep = find_sufficient_mers(df.columns[17:81]) + find_sufficient_mers(df.columns[81:337])

to_delete = pd.Index(np.setdiff1d(df.columns[17:337], to_keep))

df2 = df2.drop(to_delete, axis='columns')

df2.shape

(111897, 201)

In [322]:
df2.columns[17:49]

Index(['AAA', 'AAC', 'AAG', 'AAT', 'ACA', 'ACC', 'ACG', 'ACT', 'AGA', 'AGC',
       'AGG', 'ATA', 'ATC', 'ATG', 'CAA', 'CAC', 'CAG', 'CCA', 'CCC', 'CCG',
       'CGA', 'CGC', 'CTA', 'CTC', 'GAA', 'GAC', 'GCA', 'GCC', 'GGA', 'GTA',
       'TAA', 'TCA'],
      dtype='object')

In [323]:
df2.columns[49:185]

Index(['AAAA', 'AAAC', 'AAAG', 'AAAT', 'AACA', 'AACC', 'AACG', 'AACT', 'AAGA',
       'AAGC',
       ...
       'TAAA', 'TACA', 'TAGA', 'TATA', 'TCAA', 'TCCA', 'TCGA', 'TGAA', 'TGCA',
       'TTAA'],
      dtype='object', length=136)

In [388]:
df2.columns[185:199]

Index(['rnafold_37_energy', 'rnafold_37_frequence', 'rnafold_37_diversity',
       'rnafold_37_stem', 'rnafold_37_stem_diff', 'rnafold_37_loop',
       'rnafold_37_stem_loop_nbr', 'rnafold_75_energy', 'rnafold_75_frequence',
       'rnafold_75_diversity', 'rnafold_75_stem', 'rnafold_75_stem_diff',
       'rnafold_75_loop', 'rnafold_75_stem_loop_nbr'],
      dtype='object')

In [13]:
three_mers = np.arange(17, 49).tolist()
four_mers = np.arange(49, 185).tolist()
rnafold = np.arange(185, 199).tolist()
features = [three_mers, four_mers, rnafold]
combs = {
    0: four_mers,
    1: rnafold,
    2: four_mers  + rnafold,
}

In [390]:
cond = df2['el4-learn'] == 1

In [330]:
np.unique(df2.cas_prox_class[df['el4-learn'] == 1], return_counts=True)

(array(['CAS', 'CAS-TypeIA', 'CAS-TypeIB', 'CAS-TypeIC', 'CAS-TypeID',
        'CAS-TypeIE', 'CAS-TypeIF', 'CAS-TypeIIA', 'CAS-TypeIIB',
        'CAS-TypeIIC', 'CAS-TypeIIIA', 'CAS-TypeIIIB', 'CAS-TypeIIIC',
        'CAS-TypeIIID', 'CAS-TypeIU', 'CAS-TypeIV', 'CAS-TypeVA',
        'CAS-TypeVB', 'CAS-TypeVIA', 'CAS-TypeVIB1', 'CAS-TypeVIB2',
        'CAS-TypeVIC'], dtype=object),
 array([ 291,  450, 1505, 1637,  217, 9087, 2157,  986,   52, 1179, 1318,
         688,   46,  419,  313,   17,   42,   14,    4,  106,    4,    2]))

In [391]:
models = []
oob_res = []
for i in range(3):
    model = RandomForestClassifier(n_estimators=300, oob_score=True, n_jobs=-1, random_state=0)
    x, y = np.array(df2[cond].iloc[:, combs[i]]), df2.cas_prox_class[cond].values
    model.fit(x, y)
    models.append(model)
    oob_res.append(model.oob_score_)
np.round(oob_res, 4)

array([0.9328, 0.913 , 0.9335])

In [364]:
model = RandomForestClassifier(n_estimators=300, oob_score=True, n_jobs=-1, random_state=0)
x, y = np.array(df2[cond].iloc[:, combs[2]]), df2.cas_prox_class[cond].values
model.fit(x, y)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=300,
                       n_jobs=-1, oob_score=True, random_state=0, verbose=0,
                       warm_start=False)

### Christine's Data Sets

#### B2F05

In [409]:
data1 = pd.read_csv("data/20210415_cp_meta_b2f05_processed.csv", decimal=',', sep=';')

In [410]:
move_to_end(data1, "rnafold_37_dot_par")
move_to_end(data1, "rnafold_75_dot_par")

# convert to float
data1.iloc[:, 4:338] = data1.iloc[:, 4:338].astype('float')

# re-organize columns s.t. 3mers and 4mers are in alphabetical order
info = data1.columns[:4].tolist()
sorted_3mers = sorted(data1.columns[4:68])
sorted_4mers = sorted(data1.columns[68:324])
rnafolds = data1.columns[324:].tolist()

data1 = data1[info+sorted_3mers+sorted_4mers+rnafolds]

In [411]:
n_obs = int(data1.shape[0] / 2)
data1p = pd.concat([canonical_mers(data1.iloc[i], data1.iloc[i+n_obs], 4, 324) for i in range(n_obs)], axis=1).T

In [414]:
data1p[['ATTA', 'TAAT']]

,ATTA,TAAT
121,0,0
122,0,0
123,0,0
124,0,0
4,0,0
...,...,...
116,0.04,0.04
238,0.0285714,0.0285714
118,0.0714286,0.0714286
240,0.0294118,0.0294118


In [415]:
to_keep = find_sufficient_mers(data1p.columns[4:68]) + find_sufficient_mers(data1p.columns[68:324])

to_delete = pd.Index(np.setdiff1d(data1p.columns[4:324], to_keep))

data1p = data1p.drop(to_delete, axis='columns')

data1p.shape

(121, 188)

In [416]:
x_data1p = np.array(data1p.iloc[:, 36:186])
probs_data1p = model.predict_proba(x_data1p)
pred_data1p = model.predict(x_data1p)

#### B2F10

In [418]:
data2 = pd.read_csv("data/20210415_cp_meta_b2f10_processed.csv", decimal=',', sep=';')

In [419]:
move_to_end(data2, "rnafold_37_dot_par")
move_to_end(data2, "rnafold_75_dot_par")

# convert to float
data2.iloc[:, 4:338] = data2.iloc[:, 4:338].astype('float')

# re-organize columns s.t. 3mers and 4mers are in alphabetical order
info = data2.columns[:4].tolist()
sorted_3mers = sorted(data2.columns[4:68])
sorted_4mers = sorted(data2.columns[68:324])
rnafolds = data2.columns[324:].tolist()

data2 = data2[info+sorted_3mers+sorted_4mers+rnafolds]

In [420]:
data2.shape

(136, 340)

In [422]:
n_obs = int(data2.shape[0] / 2)
data2p = pd.concat([canonical_mers(data2.iloc[i], data2.iloc[i+n_obs], 4, 324) for i in range(n_obs)], axis=1).T

In [424]:
data2p.shape

(68, 340)

In [425]:
data2p[['ATTA', 'TAAT']]

,ATTA,TAAT
68,0.0434783,0.0434783
69,0,0
70,0,0
3,0.0285714,0.0285714
72,0,0
...,...,...
131,0.0294118,0.0294118
64,0,0
65,0.0588235,0.0588235
66,0,0


In [427]:
to_keep = find_sufficient_mers(data2p.columns[4:68]) + find_sufficient_mers(data2p.columns[68:324])

to_delete = pd.Index(np.setdiff1d(data2p.columns[4:324], to_keep))

data2p = data2p.drop(to_delete, axis='columns')

data2p.shape

(68, 188)

In [428]:
x_data2p = np.array(data2p.iloc[:, 36:186])
probs_data2p = model.predict_proba(x_data2p)
pred_data2p = model.predict(x_data2p)

In [431]:
# data1p.to_csv("data/christine_b2f05.gz", compression="gzip", index=False)
# data2p.to_csv("data/christine_b2f10.gz", compression="gzip", index=False)

In [7]:
data1p = pd.read_csv("data/christine_b2f05.gz")
data2p = pd.read_csv("data/christine_b2f10.gz")
data = pd.concat((data1p, data2p))

### Learning

In [5]:
from open_world_self_learning import SemisupProbabilisticRandomForestClassifier, OpenWorldSelfLearning

In [6]:
cond = df2['el4-learn'] == 1
x_l, y_l = np.array(df2[cond].iloc[:, 49:199]), np.array(df2.cas_prox_class[cond], dtype='<U20')
x_u = np.array(data.iloc[:, 36:186])

In [7]:
# a fast test

In [8]:
base_model = SemisupProbabilisticRandomForestClassifier(n_estimators=1, 
                                                        splitter='random', cython=True,
                                                        criterion='gini', n_jobs=-1,
                                                        max_depth=12, oob_score=False,
                                                        num_rand_splits=10,
                                                        min_samples_split=5,
                                                        max_features='sqrt',
                                                        lam=0.5, random_state=0)

model = OpenWorldSelfLearning(base_estimator=base_model, theta=0.8, gamma_rej=0.55, n_neighbors=5,
                                              min_cluster_samples=4, decreased_pl_weights=False,
                                              max_iter=2, random_state=0)

In [9]:
model.fit(x_l, y_l, x_u)

/home/vasya/Documents/crispr-dr-pred/open_world_self_learning/semisup_probabilistic_classifier.py:401: RuntimeWarning: invalid value encountered in true_divide
  (y_l_.shape[1], y_l_.shape[0])).T
/home/vasya/Documents/crispr-dr-pred/open_world_self_learning/semisup_probabilistic_classifier.py:401: RuntimeWarning: invalid value encountered in true_divide
  (y_l_.shape[1], y_l_.shape[0])).T
/home/vasya/Documents/crispr-dr-pred/open_world_self_learning/semisup_probabilistic_classifier.py:401: RuntimeWarning: invalid value encountered in true_divide
  (y_l_.shape[1], y_l_.shape[0])).T


In [10]:
# to access predicted values for x_u
model.y_u_pred

array(['-1', '-1', 'CAS-TypeIIA', '-1', '23', '-1', '-1', 'CAS-TypeIIC',
       'CAS-TypeIIIB', '-1', 'CAS-TypeIIID', 'CAS-TypeIIIB', '-1', '-1',
       '-1', '22', 'CAS-TypeIF', '-1', '-1', '-1', 'CAS-TypeVIA',
       'CAS-TypeIE', '-1', '-1', '23', '-1', '-1', 'CAS-TypeVIB1', '-1',
       '-1', 'CAS-TypeIE', '-1', 'CAS-TypeIE', 'CAS-TypeIC', '22',
       'CAS-TypeIIC', '-1', '-1', '-1', '23', '-1', 'CAS-TypeVA',
       'CAS-TypeIV', 'CAS-TypeIIC', '-1', 'CAS-TypeIE', '22', '-1',
       'CAS-TypeIB', '-1', '-1', 'CAS-TypeIE', 'CAS-TypeVIB2',
       'CAS-TypeVIB2', '-1', 'CAS-TypeVIB1', '-1', '-1', 'CAS-TypeIB',
       '23', '-1', 'CAS-TypeIB', 'CAS-TypeIC', '22', '-1', '-1', '-1',
       '-1', '-1', 'CAS-TypeIIIA', 'CAS-TypeVIA', '22', 'CAS-TypeIIIA',
       '-1', '-1', '23', 'CAS-TypeIC', '-1', 'CAS-TypeIE', '-1', '-1',
       'CAS-TypeIB', 'CAS-TypeIC', 'CAS', '-1', '-1', 'CAS-TypeVA', '-1',
       '-1', '-1', 'CAS-TypeIIC', 'CAS-TypeIIC', '-1', '-1', '-1', '-1',
       'CAS-TypeIB'

Last time I've sent results for the Christine data sets, I used the following params

In [ ]:
base_model = SemisupProbabilisticRandomForestClassifier(n_estimators=200, 
                                                        splitter='random', cython=True,
                                                        criterion='gini', n_jobs=-1,
                                                        max_depth=12, oob_score=False,
                                                        num_rand_splits=10,
                                                        min_samples_split=5,
                                                        max_features='sqrt',
                                                        lam=0.2, random_state=0)

model2 = OpenWorldSelfLearning(base_estimator=base_model, theta=0.5, gamma_rej=0.5, n_neighbors=10,
                                              min_cluster_samples=2, decreased_pl_weights=False,
                                              max_iter=1, n_jobs=1, random_state=0)